# Mengimport modul dan membuat session

In [1]:
from pyspark.sql import SparkSession
#mengimport modul ALS dari Spark Machine Learning
from pyspark.ml.recommendation import ALS

#membuat session
appName = "Sistem Rekomender di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Memuat data dari file

In [9]:
#membuat data file ke DataFrame
ratings = spark.read.csv('dataset/ratings.csv', inferSchema=True, header=True)
movies = spark.read.csv('dataset/movies.csv', inferSchema=True, header=True)
#menggabungkan data movie dan rating berdasarkan "movieId"
ratings.join(movies, "movieId").show(9)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
|   1129|     1|   2.0|1260759185|Escape from New Y...|Action|Adventure|...|
|   1172|     1|   4.0|1260759205|Cinema Paradiso (...|               Drama|
|   1263|     1|   2.0|1260759151|Deer Hunter, The ...|           Drama|War|
|   1287|     1|   2.0|1260759187|      Ben-Hur (1959)|Action|Adventure|...|
|   1293|     1|   2.0|1260759148|       Gandhi (1982)|               Drama|
|   1339|     1|   3.5|1260759125|Dracula (Bram Sto...|Fantasy|Horror|Ro...|
+-------+------+------+----------+--------------------+--------------------+

# Menyiapkan data

In [3]:
#memilih kolom data "userId", "movieId", dan "rating"
data = ratings.select("userId", "movieId", "rating")
#membagi data, 70% training dan 30% testing
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")
#menghitung baris data training dan testing
train_rows = train.count()
test_rows = test.count()
print ("Jumlah baris data training:", train_rows, 
       ", jumlah baris data testing:", test_rows)

Jumlah baris data training: 69883 , jumlah baris data testing: 30121


# Mendefiniskan model dan mentrainingnya

In [4]:
#mendefinisikan algoritma ALS untuk sistem rekomender kita
als = ALS(maxIter=19, regParam=0.01, userCol="userId", 
          itemCol="movieId", ratingCol="label")
#mentraining model dengan fungsi ".fit()"
model = als.fit(train)
print("Model telah selesai ditraining!")

Model telah selesai ditraining!


# Melakukan prediksi dengan model rekomender yang telah kita training

In [10]:
prediction = model.transform(test)
prediction.join(movies, "movieId").select(
    "userId", "title", "prediction", "trueLabel").show(n=4, truncate=False)

+------+--------------------------------+----------+---------+
|userId|title                           |prediction|trueLabel|
+------+--------------------------------+----------+---------+
|463   |Galaxy Quest (1999)             |3.7362113 |3.0      |
|243   |Men in Black (a.k.a. MIB) (1997)|3.335907  |3.0      |
|243   |Children of the Corn (1984)     |2.1835654 |2.0      |
|243   |Galaxy Quest (1999)             |3.1352015 |2.5      |
+------+--------------------------------+----------+---------+
only showing top 4 rows



# Mengevaluasi seberapa akurat sistem rekomender kita

In [11]:
#import RegressionEvaluator karena kita ingin menghitung RMSE
#, yg formulanya sama saja meski untuk sistem rekomender
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): nan


In [12]:
prediction.count()
a = prediction.count()
print("jumlah baris sebelum di hapus data kosong: ", a)
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("jumlah baris setelah di hapus data kosong: ", b)
print("jumlah baris data kosong: ", a-b)

jumlah baris sebelum di hapus data kosong:  30121
jumlah baris setelah di hapus data kosong:  28882
jumlah baris data kosong:  1239


In [8]:
rmse = evaluator.evaluate(cleanPred)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 1.230282987462192
